In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
%cd gdrive/MyDrive/grapes_project/samples/uvas/

[Errno 2] No such file or directory: 'gdrive/MyDrive/grapes_project/samples/uvas/'
/content/gdrive/MyDrive/grapes_project/samples/uvas


In [37]:
ROOT_PATH = 'stuff/pickles/dcomplete_pgrapes_aug/'
LOCATION_PICKLES = ROOT_PATH + 'location_pickles'
PREDICTION_PICKLES = ROOT_PATH + 'prediction_pickles'
JSON_PATH = 'stuff/json/ConchayToro.json'

In [30]:
import os
import pandas as pd

In [36]:
import shutil
import pickle
import json

In [32]:
def location_pickle_to_df(file_name):
    #get the hilera id
    n_hilera = file_name.split("_")[1]
    
    #Read the pickle as dict
    df = pd.read_pickle(os.path.join(LOCATION_PICKLES,file_name))
    
    #Transform to pickle to dataframe
    df = pd.DataFrame(df.items(), columns=['id_racimo', 'info_racimo'])
    df['hilera_id'] = n_hilera
    
    #Split the info racimo and concat in a new data frame
    tmp = pd.DataFrame(df['info_racimo'].tolist(), index=df.index, columns=['frame_count','x_center','distance'])
    df = pd.concat([df['hilera_id'],df['id_racimo'],tmp],axis=1)
    
    return df

In [57]:
def create_location_pickles(campo, cuartel, n_hileras, source):
  for i in range(1, n_hileras+1):
    shutil.copy(os.path.join(LOCATION_PICKLES, source), os.path.join(LOCATION_PICKLES, f'locations_{campo}_{cuartel}_{i}_1.pkl'))
    shutil.copy(os.path.join(LOCATION_PICKLES, source), os.path.join(LOCATION_PICKLES, f'locations_{campo}_{cuartel}_{i}_2.pkl'))

In [62]:
def save2pickle(df, filename):
  records = df.to_records(index=False, column_dtypes={'racimo_id': 'O',  'racimo_area': 'O'})
  result = list(records)
  tuples = list(map(tuple, result))
  with open(filename, 'wb') as f:
    pickle.dump(tuples, f)


In [63]:
def create_prediction_pickles(df, campo, cuartel, n_hileras):
  for i in range(1, n_hileras+1):
    df = df.assign(campo_id=campo)
    df = df.assign(cuartel_id=cuartel)
    df = df.assign(hilera_id=i)
    df = df.assign(am_pm=1)

    save2pickle(df, os.path.join(PREDICTION_PICKLES, f'prediction_{campo}_{cuartel}_{i}_1.pkl') )

    df = df.assign(am_pm=2)

    save2pickle(df, os.path.join(PREDICTION_PICKLES, f'prediction_{campo}_{cuartel}_{i}_2.pkl') )


In [64]:
source_lt = os.path.join(LOCATION_PICKLES, 'template.pkl')
source_pd = os.path.join(PREDICTION_PICKLES, 'template.pkl')
df = pd.read_pickle(source_pd)
df = pd.DataFrame(df, columns = ["campo_id","cuartel_id","hilera_id","am_pm","racimo_id","racimo_area"])

with open(JSON_PATH) as json_file:
    data = json.load(json_file)

for campo_id, cuarteles in data.items():
  for cuartel_id, info in cuarteles.items():
    create_location_pickles(campo_id, cuartel_id, info['n_hileras'], 'template.pkl')
    create_prediction_pickles(df, campo_id, cuartel_id, info['n_hileras'])

